In [1]:
import librosa
import numpy as np
import os
import math
import pickle
from sklearn.cluster import KMeans
import hmmlearn.hmm

In [2]:
def get_mfcc(file_path):
  y, sr = librosa.load(file_path) # read .wav file
  hop_length = math.floor(sr*0.010) # 10ms hop
  win_length = math.floor(sr*0.025) # 25ms frame
  # mfcc is 12 x T matrix
  mfcc = librosa.feature.mfcc(
      y, sr, n_mfcc=12, n_fft=1024,
      hop_length=hop_length, win_length=win_length)
  # substract mean from mfcc --> normalize mfcc
  mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
  # delta feature 1st order and 2nd order
  delta1 = librosa.feature.delta(mfcc, order=1)
  delta2 = librosa.feature.delta(mfcc, order=2)
  # X is 36 x T
  X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
  # return T x 36 (transpose of X)
  return X.T # hmmlearn use T x N matrix

In [3]:
def get_class_data(data_dir):
  files = os.listdir(data_dir)
  mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
  return mfcc

In [4]:
def clustering(X, n_clusters=20):
  kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
  kmeans.fit(X)
  print("centers", kmeans.cluster_centers_.shape)
  return kmeans  

In [36]:
class_names = ["toi", "nguoi", "dich",  "theo", "benh_nhan"]
dataset = {}
train_dataset = {}
for cname in class_names:
  dataset[cname] = get_class_data(os.path.join("data", cname))
  if cname[:4] != "test":
#         print(f"Load {cname} dataset to train")
    train_dataset[cname] = get_class_data(os.path.join("data", cname))

  # Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in train_dataset.items()], axis=0)
print("vectors", all_train_vectors.shape)
  # Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)

models = {}
for cname in class_names:
  print("====", cname)
  class_vectors = dataset[cname]
    # convert all vectors to the cluster index
    # dataset['cname'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
  dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])
    
    
  if cname == "toi":
    # =================================================================
    # toi |sil|~|t|~|o|~|i|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, random_state=0, n_iter=1000, init_params='e', params='ste', verbose=True
    ) 
    hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])

#     hmm = hmmlearn.hmm.MultinomialHMM(
#       n_components=9, init_params='e', params='ste', verbose=True
#     ) 
#     hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
#     hmm.transmat_ = np.array([
#         [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
#       ])

  elif cname == "nguoi":
    # =================================================================
    # nguoi |sil|~|ng|~|uo|~|i|~|sil|
#     hmm = hmmlearn.hmm.MultinomialHMM(
#       n_components=11, init_params='e', params='ste', verbose=True
#     )
#     hmm.startprob_ = np.array([0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
#     hmm.transmat_ = np.array([
#         [0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
#       ])
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=12, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

  elif cname == "dich":
    # =================================================================
    # dich |sil|~|d|~|i|~|ch|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, random_state=0, n_iter=1000, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])


  elif cname == "theo":
    # =================================================================
    # theo |sil|~|th|~|e|~|o|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=9, random_state=0, n_iter=1000, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.6, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])


  elif cname == "benh_nhan":
    # =================================================================
    # benh_nhan |sil|~|b|~|e|~|nh|~|silent|~|nh|~|a|~|n|~|sil|  
    hmm = hmmlearn.hmm.MultinomialHMM(
        n_components=23, random_state=0, n_iter=1000, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
    ])

  if cname[:4] != "test":
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
#       print("training class", cname)
#       print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
#       print(hmm.transmat_)
    models[cname] = hmm
    with open(os.path.join("models", cname + ".pkl"), "wb") as file: 
      pickle.dump(models[cname], file)
      print("Training done. Model has been dump to ", os.path.join("models", cname + ".pkl"))
print("Training Done")

/home/apple/anaconda3/envs/voice/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


vectors (29941, 36)
centers (20, 36)
centers (20, 36)
==== toi


         1      -12618.7346             +nan
         2       -9559.5890       +3059.1455
         3       -8256.4756       +1303.1134
         4       -7830.6025        +425.8731
         5       -7678.4578        +152.1447
         6       -7596.3506         +82.1072
         7       -7537.9307         +58.4200
         8       -7502.1063         +35.8244
         9       -7483.4973         +18.6090
        10       -7462.3849         +21.1124
        11       -7420.5560         +41.8289
        12       -7372.0105         +48.5456
        13       -7335.6191         +36.3914
        14       -7297.3025         +38.3166
        15       -7225.6105         +71.6920
        16       -7171.9298         +53.6807
        17       -7139.6455         +32.2842
        18       -7120.3916         +19.2539
        19       -7101.9216         +18.4700
        20       -7082.4747         +19.4470
        21       -7071.3992         +11.0755
        22       -7065.3947          +6.0045
        23

Training done. Model has been dump to  models/toi.pkl
==== nguoi


         1      -16207.7455             +nan
         2      -10939.4866       +5268.2589
         3       -9505.9488       +1433.5378
         4       -8981.9445        +524.0043
         5       -8811.5810        +170.3635
         6       -8718.1838         +93.3972
         7       -8636.4100         +81.7738
         8       -8537.7101         +98.6999
         9       -8415.2706        +122.4395
        10       -8363.8998         +51.3708
        11       -8334.5774         +29.3224
        12       -8266.0960         +68.4814
        13       -8250.0272         +16.0688
        14       -8221.6973         +28.3299
        15       -8202.5936         +19.1037
        16       -8197.5010          +5.0926
        17       -8195.9132          +1.5879
        18       -8193.9509          +1.9623
        19       -8191.2969          +2.6540
        20       -8189.2602          +2.0367
        21       -8187.1344          +2.1258
        22       -8182.7682          +4.3661
        23

...

         1      -14192.1547             +nan
         2      -11385.0854       +2807.0694
         3       -9983.9413       +1401.1440
         4       -9592.6965        +391.2449
         5       -9453.9016        +138.7949
         6       -9393.1079         +60.7937
         7       -9340.1825         +52.9254
         8       -9272.5171         +67.6654
         9       -9209.5211         +62.9960
        10       -9139.7613         +69.7598
        11       -9063.4370         +76.3243
        12       -9002.2288         +61.2082
        13       -8902.3676         +99.8612
        14       -8844.2563         +58.1113
        15       -8823.8607         +20.3956
        16       -8820.1397          +3.7210
        17       -8815.3967          +4.7430
        18       -8804.9107         +10.4860
        19       -8796.0209          +8.8898
        20       -8782.0640         +13.9569
        21       -8762.7489         +19.3151
        22       -8749.5973         +13.1515
        23

Training done. Model has been dump to  models/theo.pkl
==== benh_nhan


         1      -29113.7431             +nan
         2      -20064.7182       +9049.0249
         3      -17295.5385       +2769.1797
         4      -16027.1239       +1268.4146
         5      -15554.1689        +472.9550
         6      -15395.7747        +158.3942
         7      -15303.5720         +92.2027
         8      -15236.6224         +66.9496
         9      -15186.2865         +50.3359
        10      -15142.8967         +43.3898
        11      -15100.9303         +41.9664
        12      -15058.6950         +42.2353
        13      -15036.6322         +22.0627
        14      -15025.4501         +11.1821
        15      -15016.1514          +9.2987
        16      -15008.0196          +8.1318
        17      -15002.0370          +5.9826
        18      -14997.2012          +4.8358
        19      -14993.8550          +3.3461
        20      -14990.3987          +3.4564
        21      -14985.7231          +4.6756
        22      -14978.8633          +6.8598
        23

Training done. Model has been dump to  models/benh_nhan.pkl
Training Done


        66      -14946.0500          +0.0060


In [37]:
#TEST
class_tests = ["test_toi", "test_nguoi", "test_theo", "test_dich", "test_benh_nhan"]
print("Testing and Labeling")
testset = {}
for name in class_tests:
  testset[name] = get_class_data(os.path.join("data", name))

  # Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in testset.items()], axis=0)
print("vectors", all_train_vectors.shape)
  # Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)

for name in class_tests:
  testset[name] = list([kmeans.predict(v).reshape(-1,1) for v in testset[name]])

for true_cname in class_tests:
  if true_cname[:4] == "test":
    print("==================================")
    print(true_cname)
    print("==================================")

    lname = true_cname[5:]
    totalWord = 0
    true = 0
    accuracy = 0

    for O in testset[true_cname]:
      totalWord += 1
      scores = {}
      for cname, model in models.items():
        if cname[:4] != "test":
          score = model.score(O, [len(O)])
          scores[cname] = score
#     print(scores)
      srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
#     print(srt[0])

      if srt[0][0] == lname:
        true += 1
    accuracy = true/totalWord
    print("--------------------------------------------")
    print("!note: test_folder must contain wavs that it records exactly the word which be trained")
    print("accuracy: ", accuracy, ",true: ", true, ",total_word: ", totalWord)

Testing and Labeling


/home/apple/anaconda3/envs/voice/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


vectors (3262, 36)
centers (20, 36)
centers (20, 36)
test_toi
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.2 ,true:  4 ,total_word:  20
test_nguoi
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.0 ,true:  0 ,total_word:  20
test_theo
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.5 ,true:  10 ,total_word:  20
test_dich
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.05 ,true:  1 ,total_word:  20
test_benh_nhan
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.0 ,true:  0 ,total_word:  18
